<h2>Importing Libraries and Modules</h2>

In [2]:
# !pip install geopy
# !pip install scikit-learn==0.22.1 --user
# !pip install imblearn --user
# !pip3 install xgboost


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import numpy as np

from geopy.distance import great_circle
import datetime
import math
from collections import Counter

from sklearn.preprocessing import MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, KNNImputer
from sklearn.impute import MissingIndicator

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

from xgboost import XGBClassifier
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve, RandomizedSearchCV


<h2>Loading the Datasets</h2>

In [0]:
## Loading the Datasets

# train = pd.read_csv('../../data/weka_processed/train_pro.csv')
# test = pd.read_csv('../../data/test.csv')

train = pd.read_csv('drive/My Drive/ML/data/weka_processed/train_pro.csv')
test = pd.read_csv('drive/My Drive/ML/data/test.csv')


<h2>Basic Intuition on the Data</h2>

In [228]:
train.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare,label
0,189123628,10.5,834,56,0,64,'11/1/2019 0:20','11/1/2019 0:34',6.86252,79.8993,6.90330,79.8783,270.32,correct
1,189125358,10.5,791,47,0,134,'11/1/2019 0:56','11/1/2019 1:09',6.88589,79.8984,6.91373,79.8923,197.85,correct
2,189125719,10.5,1087,80,0,61,'11/1/2019 1:08','11/1/2019 1:26',6.90839,79.8651,6.93669,79.9146,301.64,correct
3,189127273,10.5,598,271,15.6638,68,'11/1/2019 2:27','11/1/2019 2:37',6.92570,79.8895,6.92748,79.8971,82.3,correct
4,189128020,?,?,?,?,?,'11/1/2019 3:34','11/1/2019 3:51',6.87441,79.8615,6.84478,79.9290,358.39,correct


In [229]:
test.head()

,tripid,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
0,213284604,10.5,924,42,2.4486,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27
1,213286352,10.5,4249,20,0.0000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70
2,213293973,10.5,1552,255,2.6588,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00
3,213294622,10.5,462,16,0.0000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32
4,213298687,10.5,814,392,12.3692,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47


In [230]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15327 entries, 0 to 15326
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   tripid                     15327 non-null  int64  
 1   additional_fare            15327 non-null  object 
 2   duration                   15327 non-null  object 
 3   meter_waiting              15327 non-null  object 
 4   meter_waiting_fare         15327 non-null  object 
 5   meter_waiting_till_pickup  15327 non-null  object 
 6   pickup_time                15327 non-null  object 
 7   drop_time                  15327 non-null  object 
 8   pick_lat                   15327 non-null  float64
 9   pick_lon                   15327 non-null  float64
 10  drop_lat                   15327 non-null  float64
 11  drop_lon                   15327 non-null  float64
 12  fare                       15327 non-null  object 
 13  label                      15327 non-null  obj

In [231]:
train.shape

(15327, 14)

In [232]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8576 entries, 0 to 8575
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   tripid                     8576 non-null   int64  
 1   additional_fare            8576 non-null   float64
 2   duration                   8576 non-null   int64  
 3   meter_waiting              8576 non-null   int64  
 4   meter_waiting_fare         8576 non-null   float64
 5   meter_waiting_till_pickup  8576 non-null   int64  
 6   pickup_time                8576 non-null   object 
 7   drop_time                  8576 non-null   object 
 8   pick_lat                   8576 non-null   float64
 9   pick_lon                   8576 non-null   float64
 10  drop_lat                   8576 non-null   float64
 11  drop_lon                   8576 non-null   float64
 12  fare                       8576 non-null   float64
dtypes: float64(7), int64(4), object(2)
memory usage:

In [233]:
test.shape

(8576, 13)

<h2>Cleaning the Data</h2>

In [0]:
## cleaning weka added unnecessary values

train = train.replace({'?': np.nan})


In [0]:
## converting label values into 0,1 instead of correct, incorrect

train.label = train.label.map(dict(correct=1, incorrect=0))

In [236]:
## check for missing values in the dataset

# train.isna().head()
train.isna().sum()

tripid                         0
additional_fare              196
duration                     196
meter_waiting                196
meter_waiting_fare           196
meter_waiting_till_pickup    196
pickup_time                    0
drop_time                      0
pick_lat                       0
pick_lon                       0
drop_lat                       0
drop_lon                       0
fare                         133
label                          0
dtype: int64

In [237]:
test.isna().sum()

tripid                       0
additional_fare              0
duration                     0
meter_waiting                0
meter_waiting_fare           0
meter_waiting_till_pickup    0
pickup_time                  0
drop_time                    0
pick_lat                     0
pick_lon                     0
drop_lat                     0
drop_lon                     0
fare                         0
dtype: int64

In [0]:
# ## fill missing values by mean of the column

# imputer = KNNImputer(n_neighbors=5, weights='uniform')
# train.iloc[:,[1,2,3,4,5,12]] = imputer.fit_transform(train.iloc[:,[1,2,3,4,5,12]])


In [239]:
train.isna().sum()

tripid                         0
additional_fare              196
duration                     196
meter_waiting                196
meter_waiting_fare           196
meter_waiting_till_pickup    196
pickup_time                    0
drop_time                      0
pick_lat                       0
pick_lon                       0
drop_lat                       0
drop_lon                       0
fare                         133
label                          0
dtype: int64

<h2>Basic Feature Engineering</h2>

In [0]:
## Join train and test datasets in order to obtain the same number of features during categorical conversion
train_len = len(train)
dataset = pd.concat(objs=[train, test], axis=0).reset_index(drop=True)


<h2>Advanced Feature Engineering</h2>

In [0]:
## new feature columns
trip_fare = []
trip_duration = []
trip_distance = []

distance_to_duration = []
fare_to_duration = []
fare_to_distance = []
waiting_fare_to_waiting_duration = []

duration_from_time = []
duration_error = []

## j48 rules
duration_bucket = []    ## 50
fare_bucket = []        ## 50

## dataset inspection rules
fare_waiting_gap = []
additional_fare_greater_than_250 = []
is_duration_zero = []

In [0]:
## iterate through each row in the dataset
for row in dataset.itertuples():
    if row.fare is not np.nan:
        fare = float(row.fare)
    else:
        fare = np.nan

    if row.additional_fare is not np.nan:
        additional_fare = float(row.additional_fare)
    else:
        additional_fare = np.nan

    if row.meter_waiting_fare is not np.nan:
        meter_waiting_fare = float(row.meter_waiting_fare)
    else:
        meter_waiting_fare = np.nan
    
    if row.meter_waiting is not np.nan:
        meter_waiting = float(row.meter_waiting)
    else:
        meter_waiting = np.nan

    if row.meter_waiting_till_pickup is not np.nan:
        meter_waiting_till_pickup = float(row.meter_waiting_till_pickup)
    else:
        meter_waiting_till_pickup = np.nan
    
    if row.duration is not np.nan:
        duration = float(row.duration)
    else:
        duration = np.nan
    
    pick_lat = row.pick_lat
    pick_lon = row.pick_lon
    drop_lat = row.drop_lat
    drop_lon = row.drop_lon
    
    pickup_time = row.pickup_time.replace('\'', '')
    drop_time = row.drop_time.replace('\'', '')
    pickup_time = pickup_time.split('/')
    drop_time = drop_time.split('/')
    
    # fare bucket
    if fare is not np.nan:
        fare_bucket.append(int(math.ceil(fare / 50)))
    else:
        fare_bucket.append(np.nan)
    
    # total fare for the trip
    if fare is not np.nan and additional_fare is not np.nan and meter_waiting_fare is not np.nan:
        cur_trip_fare = fare - (additional_fare + meter_waiting_fare)
        trip_fare.append(cur_trip_fare)
    else:
        cur_trip_fare = np.nan
        trip_fare.append(np.nan)
    
    # trip duration
    if duration is not np.nan and meter_waiting is not np.nan and meter_waiting_till_pickup is not np.nan:
        cur_trip_duration = duration - meter_waiting - meter_waiting_till_pickup
        trip_duration.append(cur_trip_duration)
        duration_bucket.append(int(math.ceil(cur_trip_duration / 50)))
    else:
        cur_trip_duration = np.nan
        trip_duration.append(np.nan)
        duration_bucket.append(np.nan)

    if(duration==np.nan):
        is_duration_zero.append(np.nan)
    elif(duration != 0):
        is_duration_zero.append(0)
    else:
        is_duration_zero.append(1)
    
    # trip distance
    cur_trip_distance = float(great_circle((drop_lat, drop_lon),(pick_lat, pick_lon)).kilometers)
    trip_distance.append(cur_trip_distance)
    
    # trip distance to duration
    try:
        if(cur_trip_duration is not np.nan):
            cur_dist_to_dur = cur_trip_distance / cur_trip_duration
        else:
            cur_dist_to_dur = np.nan
    except ZeroDivisionError:
        cur_dist_to_dur  = 0
    distance_to_duration.append(cur_dist_to_dur)
    
    # trip fare to trip duration
    try:
        if cur_trip_fare is not np.nan and cur_trip_duration is not np.nan:
            cur_fare_to_duration = cur_trip_fare / cur_trip_duration
        else:
            cur_fare_to_duration = np.nan
    except ZeroDivisionError:
        cur_fare_to_duration  = 0
    fare_to_duration.append(cur_fare_to_duration)
    
    # trip fare to trip distance
    try:
        if cur_trip_fare is not np.nan:
            cur_fare_to_distance = cur_trip_fare / cur_trip_distance
        else:
            cur_fare_to_distance = np.nan
    except ZeroDivisionError:
        cur_fare_to_distance  = 0
    fare_to_distance.append(cur_fare_to_distance)
    
    # trip waiting fare to waiting duration
    try:
        if meter_waiting_fare is not np.nan and meter_waiting is not np.nan:
            cur_waiting_fare_to_waiting_dur = meter_waiting_fare / meter_waiting
        else:
            cur_waiting_fare_to_waiting_dur = np.nan
    except ZeroDivisionError:
        cur_waiting_fare_to_waiting_dur  = 0
    waiting_fare_to_waiting_duration.append(cur_waiting_fare_to_waiting_dur)
    
    # trip fare to waiting fare
    try:
        if fare is not np.nan and meter_waiting_fare is not np.nan:
            cur_fare_waiting_gap = (fare - meter_waiting_fare) / fare
        else:
            cur_fare_waiting_gap = np.nan
    except ZeroDivisionError:
        cur_fare_waiting_gap  = 0
    fare_waiting_gap.append(cur_fare_waiting_gap)
    
    # additional fare greater than 250
    if(additional_fare is not np.nan and additional_fare > 250):
        additional_fare_greater_than_250.append(1)
    elif(additional_fare is not np.nan and additional_fare <= 250):
        additional_fare_greater_than_250.append(0)
    else:
        additional_fare_greater_than_250.append(np.nan)
    
    # time duration
    # pickup time
    month, day, y_hm = int(pickup_time[0]), int(pickup_time[1]), pickup_time[2].split(' ')
    year, h_m = int(y_hm[0]), y_hm[1].split(':')
    hour, minute = int(h_m[0]), int(h_m[1])
    pickup_time = datetime.datetime(year, month, day, hour, minute)
    
    # drop time
    month, day, y_hm = int(drop_time[0]), int(drop_time[1]), drop_time[2].split(' ')
    year, h_m = int(y_hm[0]), y_hm[1].split(':')
    hour, minute = int(h_m[0]), int(h_m[1])
    drop_time = datetime.datetime(year, month, day, hour, minute)
    
    # duration
    cur_duration_from_time = (drop_time - pickup_time).seconds
    duration_from_time.append(cur_duration_from_time)
    duration_error.append(cur_duration_from_time - duration)


In [0]:
## append new columns to dataset
dataset['trip_fare'] = trip_fare
dataset['trip_duration'] = trip_duration
dataset['trip_distance'] = trip_distance
dataset['distance_to_duration'] = distance_to_duration
dataset['fare_to_duration'] = fare_to_duration
dataset['fare_to_distance'] = fare_to_distance
dataset['waiting_fare_to_waiting_duration'] = waiting_fare_to_waiting_duration
dataset['duration_from_time'] = duration_from_time
dataset['duration_error'] = duration_error
dataset['duration_bucket'] = duration_bucket
dataset['fare_bucket'] = fare_bucket

dataset['fare_waiting_gap'] = fare_waiting_gap
dataset['additional_fare_greater_than_250'] = additional_fare_greater_than_250
dataset['is_duration_zero'] = is_duration_zero


<h2>Feature Preprocessing</h2>

In [0]:
## Scaling values into 0-1 range

scaler = MinMaxScaler(feature_range=(0,1))
features = [
    'additional_fare',
    'duration',
    'meter_waiting',
    'meter_waiting_fare',
    'meter_waiting_till_pickup',
    'pick_lat',
    'pick_lon',
    'drop_lat',
    'drop_lon',
    'fare',
    # 'trip_fare',
    # 'trip_duration',
    # 'trip_distance',
    # 'distance_to_duration',
    # 'fare_to_duration',
    # 'fare_to_distance',
    # 'waiting_fare_to_waiting_duration', 
    # 'duration_from_time',
    # 'duration_error',
    # 'duration_bucket',
    # 'fare_bucket'
]

dataset[features] = scaler.fit_transform(dataset[features])


In [0]:
## Split into training and testing again

train = dataset[:train_len]
test = dataset[train_len:]
test.drop(labels=['label'], axis=1, inplace=True)

train['label'] = train['label'].astype(int)


In [0]:
## Drop unnecessary columns
labels_to_drop = [
    'tripid',
#     'pick_lat',
#     'pick_lon',
#     'drop_lat',
#     'drop_lon',
    'pickup_time',
    'drop_time'
]

train.drop(labels=labels_to_drop, axis=1, inplace=True)


<h2>Feature Importance</h2>

In [0]:
# y = train['label']
# X = train.drop(labels=['label'], axis=1)

# # apply SelectKBest class to extract top 20 best features
# bestfeatures = SelectKBest(score_func=chi2, k='all')
# fit = bestfeatures.fit(X, y)
# dfscores = pd.DataFrame(fit.scores_)
# dfcolumns = pd.DataFrame(X.columns)

# # concat two dataframes for better visualization
# featureScores = pd.concat([dfcolumns, dfscores], axis=1)
# featureScores.columns = ['Specs', 'Score']  #naming the dataframe columns
# print(featureScores.nlargest(40, 'Score'))  #print 40 best features


In [0]:
## Drop less important columns
labels_to_drop = [
#     'trip_distance',
#     'fare_to_duration',
#     'trip_duration',
#     'distance_to_duration',
#     'duration_error',
#     'meter_waiting_till_pickup',
#     'additional_fare'
]

# train.drop(labels=labels_to_drop, axis=1, inplace=True)


<h2>Training and Evaluating Basic XGBoost Setup</h2>

In [0]:
## Seperate train features and label

y_train = train['label']
X_train = train.drop(labels='label', axis=1)

In [248]:
## model definition
model = XGBClassifier(random_state=42)

## evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_val_score(model, X_train, y_train, scoring='f1', cv=cv, n_jobs=-1)

print('Mean F1 Score: %.4f' % np.mean(scores))


Mean F1 Score: 0.9767


<h2>Handle Class Imbalancy - Weighted XGBoost</h2>

In [249]:
counter = Counter(y_train)
estimate = counter[1] / counter[0]

print(counter)
print('Estimate: %.3f' % estimate)

estimate = round(estimate)
print('Approximate Estimate: %i' % estimate)


Counter({1: 13925, 0: 1402})
Estimate: 9.932
Approximate Estimate: 10


In [250]:
## model definition
model = XGBClassifier(random_state=42, scale_pos_weight=estimate)

## evaluation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_val_score(model, X_train, y_train, scoring='f1', cv=cv, n_jobs=-1)

print('Mean F1 Score: %.4f' % np.mean(scores))


Mean F1 Score: 0.9694


<h2>Tune for the Class Weight - Weighted XGBoost GridSearchCV</h2>

In [0]:
# weights = [1, 3, 5, 7, 9, 10, 11, 12, 14, 15, 20]
# param_grid = dict(scale_pos_weight=weights)

In [0]:
# model = XGBClassifier(random_state=42)
# cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)

# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='f1')
# grid_result = grid.fit(X_train, y_train)


In [0]:
# ## report
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# # all configurations
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']

# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))


<h2>Cross Validation to Tune Hiper Parameters</h2>

In [0]:
# n_estimators = [int(x) for x in np.linspace(start=100, stop=2600, num=10)]

# max_depth = [int(x) for x in np.linspace(1, 15, num=1)]
# max_depth.append(None)

# learning_rate = [0.1, 0.01, 0.001, 0.0001]
# gamma = [0, 0.1, 1, 2, 5]
# reg_alpha = [0.01, 0.1, 1, 10]
# reg_lambda = [0.01, 0.1, 1, 10]
# booster = ['gbtree', 'gblinear', 'dart']
# tree_method = ['auto']
# num_parallel_tree = [1, 2, 5, 10]


In [0]:
# ## Creating the random grid

# param_grid = {
#     'n_estimators': n_estimators,
#     'max_depth': max_depth,
#     'learning_rate': learning_rate,
#     'gamma': gamma,
#     'reg_alpha': reg_alpha,
#     'reg_lambda': reg_lambda,
#     'booster': booster,
#     'tree_method': tree_method,
#     'num_parallel_tree': num_parallel_tree
# }

In [0]:
# model_random = XGBClassifier(random_state=42, scale_pos_weight=1, n_jobs=-1)
# cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=3, random_state=42)

# random_grid = GridSearchCV(estimator=model_random, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='f1')
# random_grid.fit(X_train, y_train)


In [0]:
# ## Best parameters
# random_grid.best_params_


In [0]:
# # ## GridSearch on best range
# param_grid = {
#     'n_estimators': n_estimators,
#     'max_depth': max_depth,
#     'learning_rate': learning_rate,
#     'gamma': gamma,
#     'reg_alpha': reg_alpha,
#     'reg_lambda': reg_lambda,
#     'booster': booster,
#     'tree_method': tree_method,
#     'num_parallel_tree': num_parallel_tree
# }

In [0]:
# model = XGBClassifier(random_state=42, scale_pos_weight=1, n_jobs=-1)
# cv = RepeatedStratifiedKFold(n_splits=4, n_repeats=3, random_state=42)

# grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='f1')
# grid.fit(X_train, y_train)


In [0]:
# grid.best_params_

<h2>Train for the Selected Setup</h2>

In [251]:
model = XGBClassifier(
    # scale_pos_weight = 1,
    scale_pos_weight = estimate,
    n_jobs = -1,
    random_state = 42,
    n_estimators = 800,
    max_depth = 15,
    learning_rate = 0.1,
    gamma = 0,
    reg_alpha = 0.1,
    reg_lambda = 0.1,
    booster = 'gbtree',
    tree_method = 'auto',
    num_parallel_tree = 1
)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)

rfc_scores = []
score = cross_val_score(model, X_train, y_train, cv=cv, scoring='f1')
rfc_scores.append(score.mean())

print('% Accuracy :', round(score.mean()*100, 4))


% Accuracy : 97.651


In [252]:
model.fit(X_train, y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=15,
              min_child_weight=1, missing=None, n_estimators=800, n_jobs=-1,
              nthread=None, num_parallel_tree=1, objective='binary:logistic',
              random_state=42, reg_alpha=0.1, reg_lambda=0.1,
              scale_pos_weight=10, seed=None, silent=None, subsample=1,
              tree_method='auto', verbosity=1)

<h2>Predicting and Preparing the Submission</h2>

In [0]:
trip_ids = test.tripid

labels_to_drop = [
    'tripid',
#     'pick_lat',
#     'pick_lon',
#     'drop_lat',
#     'drop_lon',
    'pickup_time',
    'drop_time',
#     'trip_distance',
#     'fare_to_duration',
#     'trip_duration',
#     'distance_to_duration',
#     'duration_error',
#     'meter_waiting_till_pickup',
#     'additional_fare'
]

test = test.drop(labels=labels_to_drop, axis=1)

In [254]:
predictions = model.predict(test)

# output = pd.DataFrame({'tripid': trip_ids, 'prediction': predictions})
# output.to_csv('../../submissions/160253h_submission_16.csv', index=False)
# print('Completed!')

output = pd.DataFrame({'tripid': trip_ids, 'prediction': predictions})
output.to_csv('160253h_submission_17.csv', index=False)

!cp '160253h_submission_17.csv' "drive/My Drive/ML/submissions/"

print('Completed!')


Completed!


model training.

without imputing missing values.

with cross validation (to tune hiper-parameters).

with feature engineering (added 16 new features).

datetime columns have dropped (new features added).

[model: XGBoost Classifier]

submissions:

160253h_submission_16.csv    score: 0.9784    (without class weights)

160253h_submission_17.csv    score: 0.9789    (with class weights)
